In [ ]:
pip install nselib
pip install schedule


Note: you may need to restart the kernel to use updated packages.


In [4]:
import nselib
from nselib import capital_market
import pandas as pd


In [ ]:
import os
from datetime import datetime, timedelta, date
import pandas as pd
from pandas.tseries.offsets import CustomBusinessDay
import holidays
from nselib import capital_market

# Setup
output_file = "Equity_Daily_data.csv"
in_holidays = holidays.India()
business_days = CustomBusinessDay(holidays=in_holidays)

# Date range
first_date=date(2024,1,17)
today = date.today()
diff= (today - first_date).days


now = datetime.now()
end = datetime.today() if now > now.replace(hour=9, minute=20) else datetime.today() - timedelta(days=1)
start = end - timedelta(days=diff)
dates = pd.date_range(start, end, freq=business_days)[-diff:]
date_strs = [d.strftime('%d-%m-%Y') for d in dates]

# Existing dates
existing = set()
if os.path.exists(output_file):
    existing = set(pd.read_csv(output_file, usecols=["date"])["date"].unique())

# Fetch and append new data
new_data = []
for ds in date_strs:
    if ds in existing:
        print(f"Skipping {ds}")
        continue
    try:
        df = capital_market.bhav_copy_with_delivery(ds)
        df["date"] = ds
        new_data.append(df)
        print(f"Fetched {ds}")
    except Exception as e:
        print(f"Failed {ds}: {e}")

if new_data:
    combined = pd.concat(new_data)
    combined.to_csv(output_file, mode='a', header=not os.path.exists(output_file), index=False)
    print("Data saved.")
else:
    print("No new data.")


# Convert `one_year_ago` to datetime
one_year_ago = datetime.combine(today - timedelta(days=365), datetime.min.time())

# Load full data
if os.path.exists(output_file):
    full_df = pd.read_csv(output_file, parse_dates=["date"], dayfirst=True)
    full_df["date"] = pd.to_datetime(full_df["date"], dayfirst=True)

    # Filter for the last 1 year
    last_year_df = full_df[full_df["date"] >= one_year_ago]

    # Save to a new file
    last_year_df.to_csv("Equity_Daily_data_1yr.csv", index=False)
    print("1-year data saved to 'Equity_Daily_data_1yr.csv'")

Skipping 17-01-2024
Skipping 18-01-2024
Skipping 19-01-2024
Skipping 22-01-2024
Skipping 23-01-2024
Skipping 24-01-2024
Skipping 25-01-2024
Skipping 26-01-2024
Skipping 29-01-2024
Skipping 30-01-2024
Skipping 31-01-2024
Skipping 01-02-2024
Skipping 02-02-2024
Skipping 05-02-2024
Skipping 06-02-2024
Skipping 07-02-2024
Skipping 08-02-2024
Skipping 09-02-2024
Skipping 12-02-2024
Skipping 13-02-2024
Skipping 14-02-2024
Skipping 15-02-2024
Skipping 16-02-2024
Skipping 19-02-2024
Skipping 20-02-2024
Skipping 21-02-2024
Skipping 22-02-2024
Skipping 23-02-2024
Skipping 26-02-2024
Skipping 27-02-2024
Skipping 28-02-2024
Skipping 29-02-2024
Skipping 01-03-2024
Skipping 04-03-2024
Skipping 05-03-2024
Skipping 06-03-2024
Skipping 07-03-2024
Skipping 08-03-2024
Skipping 11-03-2024
Skipping 12-03-2024
Skipping 13-03-2024
Skipping 14-03-2024
Skipping 15-03-2024
Skipping 18-03-2024
Skipping 19-03-2024
Skipping 20-03-2024
Skipping 21-03-2024
Skipping 22-03-2024
Skipping 25-03-2024
Skipping 26-03-2024


C:\Users\asus\AppData\Local\Temp\ipykernel_14172\4068090140.py:57: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  full_df = pd.read_csv(output_file, parse_dates=["date"], dayfirst=True)


In [18]:
pip install schedule


Note: you may need to restart the kernel to use updated packages.


In [4]:

import os
from datetime import datetime, timedelta, date
import pandas as pd
from pandas.tseries.offsets import CustomBusinessDay
import holidays
from nselib import capital_market

# Setup
output_file = "Index_Daily_data.csv"
in_holidays = holidays.India()
business_days = CustomBusinessDay(holidays=in_holidays)

# Date range
first_date=date(2023,1,1)
today = date.today()
diff= (today - first_date).days
end=today

start = today - timedelta(days=diff)

Indexlist = capital_market.fno_index_list()
Indexlist=Indexlist['underlying']

# Existing dates
existing = set()
if os.path.exists(output_file):
    existing = pd.read_csv(output_file)
    existing.iloc[0:0].to_csv(output_file, index=False)    
    print(f"Data delete successfully")

new_data = []

# Fetch and append new data
for indexd in Indexlist:
        try:
            df = capital_market.index_data(indexd,start.strftime("%d-%m-%Y") ,end.strftime("%d-%m-%Y"))
            new_data.append(df)
            print(f"Fetched {start.strftime("%d-%m-%y")} {end.strftime("%d-%m-%y")} {indexd}")
        except Exception as e:
            print("NoData found",indexd)
                

if new_data:
    combined = pd.concat(new_data)
    combined.to_csv(output_file, mode='a', header=not os.path.exists(output_file), index=False)
    print("Data saved.")
else:
    print("No new data.")


Data delete successfully
Fetched 01-01-23 11-07-25 Nifty 50
Fetched 01-01-23 11-07-25 Nifty BANK
Fetched 01-01-23 11-07-25 Nifty FINANCIAL SERVICES
Fetched 01-01-23 11-07-25 Nifty MIDCAP SELECT
Fetched 01-01-23 11-07-25 Nifty NEXT 50
Data saved.


In [2]:
#pip install nselib
import os
from datetime import datetime, timedelta, date
import pandas as pd
from pandas.tseries.offsets import CustomBusinessDay
import holidays
import schedule
import time
from nselib import capital_market


def fetch_and_save_index_data():
    try:
        output_file = "Index_Daily_data.csv"
        in_holidays = holidays.India()
        business_days = CustomBusinessDay(holidays=in_holidays)

        first_date = date(2024, 1, 1)
        today = date.today()
        diff = (today - first_date).days
        end = today
        start = today - timedelta(days=diff)

        Indexlist = capital_market.fno_index_list()
        Indexlist = Indexlist['underlying']

        if os.path.exists(output_file):
            pd.read_csv(output_file).iloc[0:0].to_csv(output_file, index=False)
            print(f"Data deleted successfully: {output_file}")

        new_data = []
        for indexd in Indexlist:
            try:
                df = capital_market.index_data(indexd, start.strftime("%d-%m-%Y"), end.strftime("%d-%m-%Y"))
                new_data.append(df)
                print(f"Fetched {start.strftime('%d-%m-%Y')} to {end.strftime('%d-%m-%Y')} for {indexd}")
            except Exception as e:
                print(f"No data for {indexd}: {e}")

        if new_data:
            combined = pd.concat(new_data)
            combined.to_csv(output_file, mode='a', header=not os.path.exists(output_file), index=False)
            print("✅ Data saved.")
            send_email("Index Data Fetch Success", f"Successfully fetched and saved index data on {today}")
        else:
            print("⚠️ No new data.")
            send_email("Index Data Fetch - No New Data", f"No data found for the run on {today}")

    except Exception as e:
        print(f"❌ Error: {e}")
        send_email("❌ Index Data Fetch Failed", f"An error occurred:\n\n{str(e)}")
        
print("AB")

AB
